# Worksheet 1: File locations and pre-processing
The exercises in this worksheet demonstrate some of the tools available for data analysis, and how to prepare PRECIS output for analysis (pre-processing). This can be time consuming for large amounts of data, so in this worksheet a small subset of data is used to easily demonstrate the steps involved. 
Future worksheets use data that has undergone pre-processing.

PRECIS output files are in PP format; a Met Office binary data format. This worksheet converts data to [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) format (a standard format in climate science) so that it can be used in post processing packages such as Python and the Python library [Iris](http://scitools.org.uk/iris/docs/latest/index.html).

<div class="alert alert-block alert-warning">
<b>By the end of this worksheet you should be able to:</b><br> 
- Identify and list the names of PRECIS output data in PP format using standard Linux commands.<br>
- Use basic Iris commands to load data files, and view Iris cubes. <br>
- Use Iris commands to remove the model rim, select data variables and save the output as NetCDF files.
</div>

<div class="alert alert-block alert-info">
<b>Note:</b> In the boxes where there is code or where you are asked to type code, click in the box, then press <kbd>Ctrl</kbd> + <kbd>Enter</kbd> to run the code. <br>
<b>Note:</b> An percentage sign <code>%</code> is needed to run some commands on the shell. It is noted where this is needed.<br>
<b>Note:</b> A hash <code>#</code> denotes a comment; anything written after this character does not affect the command being run. <br>
</div>

## Contents
### [1.1: Data locations and file names](#1.1) 
### [1.2: Getting started with to python and Iris](#1.2)
### [1.3: Rim removal (single file example)](#1.3)
### [1.4: Select variables](#1.4)

<a id='1.1'></a>
## 1.1 Data locations and file names
The datasets used within these worksheets are already linked to the notebook in order to provide quick and easy access for the purpose of this training. However the commands learned in this worksheet provide useful context for future work in a linux and unix scripting environment.

The dataset used in Worksheet 1 is a three year subset of monthly PRECIS output data; covering South East Asia and driven by the global climate model (GCM): HadCMQ0. It has a unique identifier code: **capha**.

**a)** Firstly, find out what location you are currently in by using the **`pwd`** command; **`pwd`** stands for **print working directory**.

In the cell below type **`%pwd`** on a new line and then press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.


In [ ]:
# Type %pwd below and press 'ctrl' + 'enter'


**b)** List the contents of this directory; **`ls`** stands for **list** and using the **`-l`** option gives a longer listing with more information, such as file size and modification date. 

In the cell below type  **`%ls`** on a separate line then press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.

In [ ]:
# Type %ls and press 'ctrl' + 'enter'.


# Type %ls -l and press 'ctrl' + 'enter'.



---
<div class="alert alert-block alert-success">
<b>Question:</b> What is the difference between <code>ls</code> and <code>ls -l</code>?  What extra information do you see? Which file was edited most recently?
</div>

<b>Answer</b>: <br>
*...Double click here to type your answer...*

---

**c)** Move to the directory (i.e. folder) called `/project/ciid/projects/PRECIS/worksheets/data/pp/cahpa`. This directory contains monthly output data in pp format from a PRECIS experiment with RUNID *cahpa*. List the contents of the directory.

**Hint:** The `cd` command stands for _change directory_

Type your command(s) below and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.


In [ ]:
# Type your %cd [directory-path] command below and press 'ctrl' + 'enter'.


# List the contents of this directory, using a previous command. 



**d)** There are a lot of files in this directory! The file names contain information on the simulated date of the data they contain - you'll learn more about the naming convention for PRECIS output in the _Data Storage System and Model Output_ presention.

For now, list only the files containing data for _September_ using the following command:

Type **`%ls *sep*`** in the code block below.

In [ ]:
# Type %ls *sep* and press 'ctrl' + 'enter'.



<div class="alert alert-block alert-info">
<b>Note:</b> The asterisk character <code>*</code> (also known as <i>glob</i>) matches any string within the filename
</div>

**e)** This still returns too many files to comfortably count manually. **`wc`** stands for **word count**; combining this command with **`ls`** allows us to count the number of items in that directory. 

In the cell below type  **`%ls *sep* | wc -l`** then press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.

In [ ]:
# Type %ls *sep* | wc -l command below and press 'ctrl' + 'enter'



---
<div class="alert alert-block alert-success">
<b>Question:</b> How many pp files are in this directory, in total?
<br>How many of these pp files contain the string 'sep'; relating to September? What command do you need to use to find this out?
</div>

<b>Answer</b>: 
<br>*Total number of pp files: 
<br>Number of September pp files:
<br>Command used to find number of september pp files:*

---

**f)** To list all the files containing data from 1982, we use the code **pmi2**.  Remember, you learn how to decode PRECIS output files names later...

Type below **`%ls cahpaa.pmi2???.pp`**

In [ ]:
# Type %ls cahpaa.pmi2???.pp and press 'ctrl' + 'enter'.



<div class="alert alert-block alert-info">
<b>Note:</b> The question mark character <code>?</code> matches any single character
</div>

**g)** Now move up two levels in the directory tree and list the directories.

Type `cd ../..` to move up two levels in the directory tree and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd> to execute the command

In [ ]:
# Type %cd ../.. and press 'ctrl' + 'enter'.



---
<div class="alert alert-block alert-success">
<b>Question:</b> Which directory are you in now?  What else can you see in this directory?
</div>

<b>Answer</b>:<br>
*...Double click here to type your answer...*



***
<a id='1.2'></a>
## 1.2 Getting started with Python and Iris
<p><img src="img/python_and_iris.png" alt="python + iris logo" style="float: center; height: 100px;"/></p>


Python is a general purpose programming language. Python supports modules and packages, which encourages program modularity and code reuse. 


We also use the Python library [Iris](http://scitools.org.uk/iris/docs/latest/index.html), which is written in Python and is maintained by the Met Office. Iris seeks to provide a powerful, easy to use, and community-driven Python library for analysing and visualising meteorological and oceanographic data sets.

The top level object in Iris is called a <b>cube</b>. A cube contains data and metadata about a phenomenon (i.e. air_temperature). Iris handles several different types of file formats, loadiing them into Iris cubes.


For a brief introduction to Iris and the cube formatting please read this Introduction page here: 

http://scitools.org.uk/iris/docs/latest/userguide/iris_cubes.html

For future reference please refer to the Iris website:

http://scitools.org.uk/iris/docs/latest/index.html


**h)** First, run the code-block below to **load** a file into Iris and **print** its metadata structure. <br>

To run the code, click in the box below and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.

In [ ]:
# import the necessary modules
import iris
import glob
import os

# this is needed so that the plots are generated inline with the code instead of a separate window
%matplotlib inline 

# provide the path of your sample data
sample_data = '/project/ciid/projects/PRECIS/worksheets/data/sample_data.nc'

# Constraint the reading to a single variable and load it into an Iris cube
cube = iris.load_cube(sample_data)

# Print the Iris cube
print(cube)

---
<div class="alert alert-block alert-success">
<b>Question:</b> Can you explain how our sample data we printed above relates to this picture?
<img src="img/multi_array_to_cube.png" alt="diagram of an Iris cube" style="height: 300px"/> <br>

- Is our data above a 3D or a 2D cube? <br>
- What are the cube dimensions? <br>
- How many grid boxes is the latitudinal range divided into? <br>
- What meteorological variable does this cube represent? <br>
- What unit is used for this variable? <br>
</div>


<b>Answer</b>:
<br>*Is this cube 3D or 2D? 
<br>What are the cube dimensions? 
<br>How many grid boxes is the latitudinal range divided into? 
<br>What meteorological variable does this cube represent? 
<br>What unit is used for this variable?* 

---

**i)** Now **plot** the data for the selected variable: <br>

To run the code, click in the box below and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.

In [ ]:
import matplotlib.pyplot as plt
import iris.quickplot as qplt

plt.figure(figsize=(12,12))  # Set the figure size
qplt.pcolormesh(cube)  # Plot the cube
plt.title(cube.name())  # Add plot title
plt.clim(0, 5e-4)  # Set colour bar range
plt.show()  # Show the plot

---
<div class="alert alert-block alert-success">
<b>Question:</b> Do you notice any irregularities in precipitation at the edges of the plot?  Do you think this is realisitc or unrealistic? 
</div>

<b>Answer</b>:<br>
*...Double click here to type your answer...*

---

<a id='1.3'></a>
## 1.3 Rim removal (single file example)

The edges (or rim) of RCM outputs are biased due to the linear relaxation used on certain variables to apply the GCM lateral boundary conditions at the RCM's finer resolution. This rim from each edge needs to be excluded from any analysis.

The practice PP files are 25km resolution and have an 8-point rim. The following code demonstrates how to remove the 8-point rim from all data in a specified pp file data directory.

---
<div class="alert alert-block alert-success">
<b>Question:</b> Read through the code below.  Identify the parts of the code that:<br>

- Sets the input data directory <br>
- Specifies the output file names<br>
</div>
---

**j) Remove the 8 point rim** from the raw pp files from 1961-1990 and save the 'rim-removed' cube in netCDF format. 

To run the code, click in the box and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd>.  This takes a little longer to run than the other code  you've run so far - wait until you see confirmation that the cubes have been saved before you move on...

In [ ]:
datadir = '/project/ciid/projects/PRECIS/worksheets/data'

rim_width = 8  # width of rim (in number of grid boxes)

for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(datadir, 'pp', runid)
    
    # find all the files from which to remove the rim
    file_list = glob.glob(ppdir + '/*pm[ghij]*.pp')
    
    cubes = iris.load(file_list)
    trimmed_cubes = []
    for cube in cubes:
        # get longitude coordinate
        xcoord = cube.coord(axis='X', dim_coords=True)
        # get latitude coordinate
        ycoord = cube.coord(axis='Y', dim_coords=True)
        # Remove rim from longitude
        rrcube = cube.subset(xcoord[rim_width:-1 * rim_width])
        # Remove rim from latitude
        rrcube = rrcube.subset(ycoord[rim_width:-1 * rim_width])
        # add meta data stating that rim has been removed
        rrcube.attributes['rim_removed'] = '{} point rim removed'.format(rim_width)
        trimmed_cubes.append(rrcube)
    rrcubes = iris.cube.CubeList(trimmed_cubes)
    # Write out the trimmed data file
    outfile = os.path.join(datadir, 'historical', runid + '.mon.1961_1990.rr.nc')
    iris.save(rrcubes, outfile)
    print('Saved {}'.format(outfile))

<div class="alert alert-block alert-info">
<b>Note:</b> iris.load() attempts to return as few cubes as possible by collecting together multiple fields with a shared standard name and sequential time steps into a single multidimensional cube. Read more about iris loading behaviour <a href="https://scitools.org.uk/iris/docs/latest/userguide/loading_iris_cubes.html">here</a>.
</div>

Take note of the file names.  Good use of filename can help you keep track of the contents of your files.  We suggest developing a consistent syntax.  Here the file names shows the following:

**cahpa.pm.1961_1990.rr.nc**

* cahpa = The run-ID of the model
* mon = monthly data
* 1961_1990 = the time period of the data
* rr = rim removed
* .nc = netCDF format

<div class="alert alert-block alert-info">
<b>Note:</b> Keep a note of how we update the file names as we progress through these worksheets making further changes to the data
</div>

---
<div class="alert alert-block alert-success">
<b>Question:</b> How many files are in the file list used by the iris.load() command? <br> How many time-steps are in the final netCDF outfile?
</div>

---

***
<a id='1.4'></a>
## 1.4 Select variables

In these worksheets, the monthly data files contain multiple variables; we can use Iris to separate the variables and save them as separate netCDF files.

**k)** **Separate each variable** into its own file.

Click in the box below and press <kbd>Ctrl</kbd> + <kbd>Enter</kbd> to run the code.

In [ ]:
#Dictionary of stash codes and associated variable names 
stash_codes = {'m01s03i236':'tm',
               'm01s05i216':'pr'}

for runid in ['cahpa', 'cahpb']:
    # Get data directory
    infile = os.path.join(datadir, 'historical', runid + '.mon.1961_1990.rr.nc')
    # This will load all the variables in the file into a CubeList
    datacubes = iris.load(infile)
    for cube in datacubes:
        # get the STASH code
        cubeSTASH = cube.attributes['STASH']
        # Make the output file name
        outfile = os.path.join(datadir, 'historical', runid + '.mon.1961_1990.' + stash_codes[str(cubeSTASH)] + '.rr.nc')
        # Save the file
        iris.save(cube, outfile)
        print('Saved {}'.format(outfile))            

<div class="alert alert-block alert-info">
    <b>Note:</b> Now that we've separated out different variables, notice that we've appended <b>tm</b> for mean surface temperature and <b>pr</b> for precipitation flux to the file names

---
<div class="alert alert-block alert-success">
<b>Question:</b> Use the <b>cd</b> and <b>ls</b> commands to check the NetCDF directory that you have been creating new files in. <br>

- Confirm the names of the new files you have been creating. <br>
- What is the size of the concatenated file (containing 1961-1990 data)?
</div>

In [ ]:
# use %cd and %ls to list the contents of your new directory containing NetCDF files:


# use %ls -lh to compare the size of the single month pp files and the concatenated netcdf file: 



<b>Answer:</b><br>
*Size of the concatenated file:*

---
<div class="alert alert-block alert-success">
<b>Extension Question:</b> How could you improve this workflow? Do we really need to save the intermediate files created in <strong>1.3</strong>?
</div>

---

<center>
<div class="alert alert-block alert-warning">
<b>This completes worksheet 1.</b> <br>You have created pre-processed files (rim removed, individual variables, concenated over time, in NetCDF format). <br>
In worksheet 2, you will begin to analyse these files.
</div>
</center>

<p><img src="img/MO_MASTER_black_mono_for_light_backg_RBG.png" alt="python + iris logo" style="float: center; height: 100px;"/></p>
<center>© Crown Copyright 2019, Met Office</center>